## test cuda available

In [1]:
 
import torch
print('torch version:',torch.__version__)
print('cuda version:',torch.version.cuda)
if torch.cuda.is_available():
    print('cuda available')
else:
    print('cuda unavailable')
print('muti device count:', torch.cuda.device_count())


torch version: 2.1.2+cu118
cuda version: 11.8
cuda available
muti device count: 4


In [6]:
import torch
input = torch.tensor([[1, 2], [3, 4], [5, 6]])
index = torch.tensor([[0, 1], [1, 2]])
# (0->1,0)(0->1,1) , (1->1,0)(1->2,1) 
output = torch.gather(input, dim=0, index=index)
print(output)

index = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 1, 0]])
# (0,0)(0,1)(0,1) , (1,0->1)(1,1->0)(1,1->0) , (2,0->1)(2,1->1)（2，2->0)
output = torch.gather(input, dim=1, index=index)
print(output)

tensor([[1, 4],
        [3, 6]])
tensor([[1, 2, 2],
        [4, 3, 3],
        [6, 6, 5]])


In [8]:
def extract(v, t, x_shape):
    """
    Extract some coefficients at specified timesteps, then reshape to
    [batch_size, 1, 1, 1, 1, ...] for broadcasting purposes.
    """
    device = t.device
    out = torch.gather(v, index=t, dim=0).float().to(device)
    print ([t.shape[0]] + [1] * (len(x_shape) - 1))
    return out.view([t.shape[0]] + [1] * (len(x_shape) - 1))
v = torch.tensor([1, 2, 3, 4, 5])
t = torch.tensor([0, 2, 4])
out = extract(v, t, x_shape=(len(t),))
print(out.shape)
print(out)

[3]
torch.Size([3])
tensor([1., 3., 5.])


In [9]:
import torch.nn.functional as F
import torch

alphas_bar = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5])
T = 3

alphas_bar_prev = F.pad(alphas_bar, [1, 0], value=1)[:T]

print(alphas_bar_prev.shape)  
print(alphas_bar_prev)        

torch.Size([3])
tensor([1.0000, 0.1000, 0.2000])


In [11]:
import torch  

n_T = 10
batch_size = 4
_ts = torch.randint(1, n_T + 1, (batch_size,))
print('_ts.shape:',_ts.shape) 

beta2 = 0.02
beta1 = 1e-4
beta_t = (beta2 - beta1) * torch.arange(0, n_T + 1, dtype=torch.float32) / n_T + beta1
sqrt_beta_t = torch.sqrt(beta_t)
alpha_t = 1 - beta_t
log_alpha_t = torch.log(alpha_t)
alphabar_t = torch.cumsum(log_alpha_t, dim=0).exp()
sqrtab = torch.sqrt(alphabar_t)
oneover_sqrta = 1 / torch.sqrt(alpha_t)
sqrtmab = torch.sqrt(1 - alphabar_t)
mab_over_sqrtmab_inv = (1 - alpha_t) / sqrtmab
print('beta_t.shape',beta_t.shape)

x = torch.randn((batch_size,3,32,32))
eps = torch.randn_like(x)
print('x.shape:',x.shape)
print('eps.shape:',eps.shape)
print('sqrtab_extract.shape',sqrtab[_ts, None, None, None].shape)
x_t = (
    sqrtab[_ts, None, None, None] * x
    + sqrtmab[_ts, None, None, None] * eps
) 
print('x_t.shape',x_t.shape)

simple_i = 5
_ti = torch.tensor(simple_i/n_T).repeat(batch_size,1) # repeat(*sizes)函数可以将一个张量在指定的维度上重复多次
print('_ti.shape:',_ti.shape,'\n_ti:',_ti)

_ts.shape: torch.Size([4])
beta_t.shape torch.Size([11])
x.shape: torch.Size([4, 3, 32, 32])
eps.shape: torch.Size([4, 3, 32, 32])
sqrtab_extract.shape torch.Size([4, 1, 1, 1])
x_t.shape torch.Size([4, 3, 32, 32])
_ti.shape: torch.Size([4, 1]) 
_ti: tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000]])


In [ ]:
!pip install datetime

In [8]:
import datetime
import os
folder_name = datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
print(folder_name)
folder_path = os.path.join("extra",folder_name)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)


2024-05-08-20:17:55
